In [83]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import os
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from uuid import uuid4
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
load_dotenv()

True

In [80]:
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT_ID = os.getenv('AZURE_OPENAI_DEPLOYMENT_ID')
AZURE_OPENAI_KEY = os.getenv('AZURE_OPENAI_KEY')
AZURE_API_VERSION = os.getenv('AZURE_API_VERSION')
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "ocbc-cv-gpt"  # change if desired
index = pc.Index(index_name)

llm = AzureChatOpenAI(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment=AZURE_OPENAI_DEPLOYMENT_ID,
            api_version=AZURE_API_VERSION,
            api_key=AZURE_OPENAI_KEY,
            temperature=0.0,
            verbose=True,
        )

embedding_llm = AzureOpenAIEmbeddings(
            azure_endpoint=AZURE_OPENAI_ENDPOINT,
            azure_deployment='embedding-ada-crayon',
            api_key=AZURE_OPENAI_KEY,
            api_version=AZURE_API_VERSION,
        )

vector_store = PineconeVectorStore(index=index, embedding=embedding_llm)
retriever = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 6})

In [81]:
cv = """IHSAN FATHYA
 081283576216 | ihsanfathya@gmail.com | https://www.linkedin.com/in/ihsan-fathya/ | https://github.com/ihsaan31
 Bogor, Jawa Barat
 I'm an 8th-semester student with a genuine passion for technology, specifically in machine learning and Data Science. Drawing from a 
solid background in computer science and programming, I've developed a keen interest in utilizing Python to address complex problems 
and create innovative solutions.
 Work Experiences
 OCBC Indonesia -  Bsd, Tangerang Jun 2024 - Present
 Data Scientist Intern
 Oversea-Chinese Banking Corporation Limited (OCBC) is a Singaporean multinational banking and financial services corporation that 
provides a range of services to individuals, businesses, government agencies, and financial institutions.
 AI Compliance Chat Bot
 Developed a robust AI-powered compliance chatbot that integrates the regulatory frameworks of OJK, BI, and Sikepo. Leveraged 
LangChain to design and implement custom AI chains tailored for various tasks, including question routing, contextualization, and 
the generation of combined responses from multiple sources.
 Designed custom prompts and templates to optimize the performance of the AI models. These include templates for contextualizing 
questions, generating system responses, and creating combined answers by synthesizing data from multiple regulatory bodies.
 Implemented retrieval systems using various index managers (ElasticSearch, Redis) and graph databases (Neo4j) to fetch and 
merge relevant information, optimizing the accuracy of AI responses.
 FastAPI Development and API Integration
 Designed and implemented a RESTful API using FastAPI, enabling efficient communication between the front-end and backend 
services.
 Implemented real-time data streaming using 'StreamingResponse', allowing continuous delivery from LangChain.
 BMKG | Badan Meteorologi, Klimatologi, dan Geofisika -  
Jakarta Pusat
 Oct 2023 - Dec 2023
 Artificial Intelligence Intern
 BMKG, based in Jakarta Pusat, is a key governmental organization in Indonesia. It plays a crucial role by providing essential information 
in meteorology, climatology, and geophysics. BMKG's services are integral to various sectors, including agriculture, transportation, 
disaster management, and public safety.
 Model Monitoring Dashboard
 Created a comprehensive web monitoring dashboard using Dash Plotly to evaluate and visualize model performance metrics 
effectively.
 Developed a separate monitoring dashboard utilizing Looker Studio, enhancing accessibility and usability.
 AI Modeling
 Applied Python extensively for AI modeling, including the conversion of an existing R model to Python resulting 10% improvment in 
accuracy.
 Developed and implemented a model for refining Numerical Weather Prediction using various machine learning algorithms, 
including Scikit-Learn and TensorFlow achieving 77% accuracy.
 PFI mega life -  Jakarta Selatan Feb 2023 - Jun 2023
 Data Analyst Intern
 PT PFI Mega Life Insurance is a joint venture with a new structure that combines the global expertise of Prudential Financial Inc. with the 
local market network strength of CT Corpora. PFI Mega Life provides a complete range of life insurance products to serve customers with 
a wide range, from large corporations to individuals.
 Leads Segmentation
 Integrated sentiment analysis and demographic results into leads segmentation, providing a nuanced understanding of leads 
engagement.
 Implemented K-means clustering algorithm for leads segmentation into distinct clusters.
 Sentiment Analysis
 Conducted sentiment analysis on a dataset of 6,000 Activity Content from CRM posts using Python and various libraries, including  
NLTK, Sastrawi, pandas, numpy, matplotlib, and seaborn.
 Utilized the pre-trained Indonesia BERT model for sentiment analysis, resulting in a 10% increase in accuracy compared to 
traditional methods.
 Applied the model to categorize Activity Content into positive, neutral, or negative sentiments.
 Scrape LinkedIn Profile
 Executed a web scraping initiative using Beautiful Soup and Selenium in Python, successfully extracting over 14,000 leads from 
LinkedIn and search engine results, exceeding the initial target by 30%.
 Successfully navigated through LinkedIn profiles to capture demographic details, encompassing education information, current job 
roles, and past experience.
 Executed data extraction techniques to compile an Excel output file, streamlining the analysis of acquired leads.
 Education Level
 Universitas Gunadarma - Depok, Jawa Barat Aug 2020 - Sep 2024 (Expected)
 Bachelor Degree in Informatics Engineering, 3.41/4.00
 Skills, Achievements & Other Experience
 Certification (2021): Machine Learning  (Dicoding)
 Certification (2022): EF SET English Certificate 78/100 (C2 Proficient)
 Certification (2023): AI-900 - Microsoft Azure AI Fundamentals
 Certification (2023): Codecademy Data Scientist: Analytics Specialist Career Path
 Certification (2023): Codecademy Data Scientist: Inference Specialist Career Path
 Certification (2024): Codecademy Data Scientist: Machine Learning Specialist Career Pat"""

In [95]:
CV_sumerrizer = """
Extract key details from a CV or resume, including name, contact information, work experience, education, skills, and certifications. Structure the output in a format optimized for a retriever system, ensuring each category is clearly labeled for easy indexing and retrieval.
 **User CV:** 
 {cv}

"""

QA_PROMPT = """
user question: {question}

if the user question is not empty, ignore the text below!
You are an AI job assistant. Based on the user's preferences, qualifications, and experience, evaluate the following job posting.

1. Mention what jobs on the current listing
2. Compare the user's CV/resume to the job listing, highlighting the key areas of alignment and any gaps.
3. Assess whether the job is a match for the user's skills and experience. If the job matches, explain why. If it does not match, clearly state: "Does not match."

CV/Resume: 
{cv}

Job Listing: 
{job_listing}

"""

In [98]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
from langchain.chains import create_history_aware_retriever

# Assuming you have defined 'llm' and 'retriever' elsewhere in your code

# CV summarizer chain
cv_summarizer_prompt = PromptTemplate.from_template(CV_sumerrizer)
cv_summarizer_chain = (
    {"cv": RunnablePassthrough()} | cv_summarizer_prompt | llm | StrOutputParser()
)

# QA chain
qa_prompt = PromptTemplate.from_template(QA_PROMPT)
qa_chain = (
    {"cv": RunnablePassthrough(),
     "job_listing": RunnablePassthrough(),
     "question": RunnablePassthrough(),} | qa_prompt | llm | StrOutputParser()
)

# Merged chain
merged_rag_chain = (
    {
        "cv": RunnablePassthrough(),
        "job_listing": RunnablePassthrough(),
        "question": RunnablePassthrough(),
    }
    | cv_summarizer_chain
    | (lambda x: {"job_listing": retriever.invoke(x), "cv": x})
    | qa_chain
    | llm
    | StrOutputParser()
)

In [99]:
merged_rag_chain.invoke({"cv": cv})

'To improve their chances of being considered for the Marketing Technology Lead or Software Engineer positions, the user could highlight their relevant experience and skills in their CV/resume. They could also consider obtaining additional certifications or training in areas that align with the job requirements. Additionally, they could tailor their application materials to emphasize their transferable skills and showcase how their previous experiences can be applied to the desired roles.'